# Sprint 4 - Analytics, Visualization, and Polish

This notebook assembles production-facing outputs:
- value/payoff chart and trigger annotation
- sensitivity heatmap for $(\mu,\sigma) \mapsto x^*$
- path overlay with crossing markers
- exportable result table for reporting

**Roadmap:** Sprint 4 (Weeks 7-8).

In [ ]:
import os
import numpy as np

from optistop import (
    GBM,
    Grid,
    PSORSolver,
    PrimalObstacleSolver,
    ResultFrame,
    VanillaCall,
    vega_from_triggers,
)
from optistop.solvers.fdm import FDMOperator
from optistop.viz.plots import plot_boundary_heatmap, plot_path_overlay, plot_value_vs_payoff

os.makedirs('output/optistop_notebooks', exist_ok=True)

In [ ]:
# Solve baseline model and export table
r, mu, sigma = 0.05, 0.02, 0.20
process = GBM(mu=mu, sigma=sigma)
utility = VanillaCall(investment_cost=1.0)
grid = Grid(x_min=1e-4, x_max=8.0, n=2000)
operator = FDMOperator(process=process, r=r)
psor = PSORSolver(omega=1.2, tol=1e-7, max_iter=50_000)

result = PrimalObstacleSolver(grid=grid, utility=utility, operator=operator, psor=psor).solve()

plot_value_vs_payoff(result, 'output/optistop_notebooks/value_vs_payoff.png')
ResultFrame(result).to_df().to_csv('output/optistop_notebooks/value_table.csv', index=False)

print('Trigger x*:', result.trigger)
print('Saved: value_vs_payoff.png, value_table.csv')

In [ ]:
# Sensitivity surface for trigger x*(mu, sigma)
mu_grid = np.linspace(0.01, 0.08, 8)
sigma_grid = np.linspace(0.10, 0.50, 9)
trigger_surface = np.zeros((sigma_grid.size, mu_grid.size))

for i, s in enumerate(sigma_grid):
    for j, m in enumerate(mu_grid):
        proc = GBM(mu=m, sigma=s)
        op = FDMOperator(process=proc, r=r)
        res = PrimalObstacleSolver(grid=grid, utility=utility, operator=op, psor=psor).solve()
        trigger_surface[i, j] = res.trigger

plot_boundary_heatmap(mu_grid, sigma_grid, trigger_surface, 'output/optistop_notebooks/boundary_heatmap.html')

vega_center = vega_from_triggers(sigma_grid, trigger_surface[:, len(mu_grid)//2])
print('Median Vega around center drift:', float(np.median(vega_center)))
print('Saved: boundary_heatmap.html')

In [ ]:
# Monte Carlo path overlay with stopping trigger
T, dt, n_paths = 3.0, 0.01, 30
paths = process.simulate_paths(T=T, dt=dt, n_paths=n_paths, x0=1.0)
time_grid = np.linspace(0, T, int(np.ceil(T / dt)) + 1)

plot_path_overlay(time_grid, paths, result.trigger, 'output/optistop_notebooks/path_overlay.png')
print('Saved: path_overlay.png')

## Sprint 4 Deliverable Check

- [x] Publication-style value vs payoff plot generated
- [x] Sensitivity heatmap generated
- [x] Path overlay with trigger crossings generated
- [x] Export-ready table produced for Chapter 4 reporting